In [ ]:
import numpy as np
import scipy as sp

from change_of_basis import tmat_of_cmat
from find_symmetry_groups import proj_to_vsig_of_u_new
from find_symmetry_groups import UsHat
from plotter import Plotter
from utilities import v2sm

In [ ]:
write = False

In [ ]:
study = "MSAT"
file_path = "../data/TX2008.V1.P010.dat"
data = np.loadtxt(file_path)
n_material = np.size(data,0)
lons = data[:n_material,1]
lats = data[:n_material,0]
cijs = data[:n_material,3:24]

In [ ]:
betas = np.load(f"Data/{study}/betas_{study}.npy")
thetas = np.load(f"Data/{study}/thetas_{study}.npy")
sigmas = np.load(f"Data/{study}/sigmas_{study}.npy")
phis = np.load(f"Data/{study}/phis_{study}.npy")
dp_m1 = sp.io.loadmat(f"Data/{study}/decomp_percentages_{study}.mat")["DP_MS_axes"]

In [ ]:
def angle(A, B):
    a = A.flatten()
    b = B.flatten()
    return np.rad2deg(np.arccos(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))))

In [ ]:
symmetry_classes = ["ISO","XISO","TET","ORTH","MONO"]
n_symmetry_classes = len(symmetry_classes)

In [ ]:
Y = np.zeros((n_symmetry_classes,n_material))
Z = np.zeros((n_symmetry_classes,n_material))
for i, sigma in enumerate(symmetry_classes[1:], start=1):
    sigma1 = "ISO"
    sigma2 = symmetry_classes[i-1]
    for j in range(n_material):
        c_vec = cijs[j,:]
        c_mat = v2sm(c_vec)
        t_mat = tmat_of_cmat(c_mat)
        A1 = proj_to_vsig_of_u_new(t_mat, UsHat([thetas[0,j], sigmas[0,j], phis[0,j]]), sigma1)
        A2 = proj_to_vsig_of_u_new(t_mat, UsHat([thetas[i-1,j], sigmas[i-1,j], phis[i-1,j]]), sigma2)
        B = proj_to_vsig_of_u_new(t_mat, UsHat([thetas[i,j], sigmas[i,j], phis[i,j]]), sigma)
        Y[i,j] = angle(A1,B)
        Z[i,j] = angle(A2,B)

In [ ]:
plot = Plotter(lons, lats, central_longitude=0, write=write)

In [ ]:
for i, sigma in enumerate(symmetry_classes[1:], start=1):
    sigma1 = "ISO"
    sigma2 = symmetry_classes[i-1]
    x = betas[i,:]
    y = Y[i,:]
    z = Z[i,:]
    xlabel = f"angle({sigma},TRIV)"    
    ylabel = f"angle({sigma1},{sigma})"
    zlabel = f"angle({sigma2},{sigma})"
    filename = f"angles_{sigma}_{study}"
    plot.multi_correlation_plots(x, y, z, xlabel, ylabel, zlabel, trend=True, aspect="equal", filename=filename)

In [ ]:
x = betas[0,:]
y = Y[1,:]
cmap1 = ["RdYlGn", False, 0.00, 4.00]
cmap2 = ["RdYlGn", False, 0.00, 3.50]
xlabel = r"$\beta_{ISO}$ (°)"
ylabel = f"angle(ISO,XISO)"
filename = f"angle_ISO_XISO_beta_ISO_{study}"
plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmap1, cmaps2=cmap2, residual=False, trend=True, aspect="equal", filename=filename)

In [ ]:
x = dp_m1[:,1] * 100
y = Y[1,:]
cmap1 = ["jet", False, 0.00, 4.00]
cmap2 = ["jet", False, 0.00, 3.50]
xlabel = f"HEX_percentage"
ylabel = f"angle(ISO,XISO)"
filename = f"angle_ISO_XISO_HEX_percentage_{study}"
plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmap1, cmaps2=cmap2, residual=False, aspect="equal", filename=filename)

correlation_matrix = np.corrcoef(x, y)
correlation_coefficient = correlation_matrix[0,1]
print(f"The correlation coefficient is {correlation_coefficient}")